# COMP4167 Natural Language Processing
# Practical III
# Transformers with Sentiment Analysis

In this notebook we will be using the transformer model, BERT (Bidirectional Encoder Representations from Transformers) model to perform sentiment analysis of movie review dataset IMDB.

In this notebook we will:
- Use the [transformers library](https://github.com/huggingface/transformers) to get pre-trained BERT model and use it as our embedding layer. 
- We will freeze (not train) the transformer and only train the model which learns from the representations produced by the transformer. 
- We will use a multi-layer bi-directional GRU, however any model can learn from these representations.

## Preparing Data

The transformer has been pre-trained with a specific vocabulary, which means we need to train with the exact same vocabulary and also tokenize our data in the same way that the transformer did when it was initially trained.

The transformers library has tokenizers for each of the transformer models provided. We are using the BERT model which ignores casing. We get this by loading the pre-trained `bert-base-uncased` tokenizer. This is one of the smaller BERT models but still contains 110 million parameters.

In [157]:
import warnings
warnings.filterwarnings('ignore')
import datasets
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from transformers import BertTokenizer, BertModel
import random

#Set the random seeds for deterministic results.
SEED = 1234

random.seed(SEED)

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [158]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

The `tokenizer` has a `vocab` attribute which contains the actual vocabulary we will be using. We can check how many tokens are in it by checking its length.

In [159]:
len(tokenizer.vocab)

30522

Using the tokenizer is as simple as calling `tokenizer.tokenize` on a string. This will tokenize and lower case the data in a way that is consistent with the pre-trained transformer model.

In [160]:
tokens = tokenizer.tokenize('Hello WORLD how ARE yoU?')
print(tokens)

['hello', 'world', 'how', 'are', 'you', '?']


We can numericalize tokens using our vocabulary using `tokenizer.convert_tokens_to_ids`.

In [161]:
indexes = tokenizer.convert_tokens_to_ids(tokens)
print(indexes)

[7592, 2088, 2129, 2024, 2017, 1029]


The transformer was also pre-trained with special tokens to mark the beginning and end of the sentence, detailed [here](https://huggingface.co/transformers/model_doc/bert.html#transformers.BertModel). As well as a standard padding and unknown token. We can also get these from the tokenizer.

- CLS = Classification token
- SEP = Seperating token
- Pad = Padding token
- UNK = Unknown token


In [162]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


We can get the indexes of the special tokens by converting them using the vocabulary...

In [163]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token) 
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


...or by explicitly getting them from the tokenizer.

In [164]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


Another thing we need to handle is that the model was pre-trained on sequences with a defined maximum length - it does not know how to handle sequences longer than it has been trained on. We can get the maximum length of these input sizes by checking the `max_model_input_sizes` for the version of the transformer we want to use. In this case, it is 512 tokens.

In [165]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

512


We will define a utility function to tokenize sentences based on the tokenizer provided by the transformers model.

In [166]:
def tokenize_and_numericalize_data(example, tokenizer):
    ids = tokenizer(example['text'], truncation=True)['input_ids']
    return {'ids': ids}

## Data split
Now we load the data and create the validation splits.
IMDB is available within torchtext.

In [167]:
train_data, test_data = datasets.load_dataset('imdb', split=['train', 'test'], ignore_verifications=True)

test_valid_data = test_data.train_test_split(test_size=0.25)
test_data = test_valid_data['train']
valid_data = test_valid_data['test']

Found cached dataset imdb (/home/sean/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/2 [00:00<?, ?it/s]

In [168]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 25000
Number of validation examples: 6250
Number of testing examples: 18750


We can check an example. We can see that the words/tokens have been now replaced by their indexes. The label for this example is negative.

In [169]:
print((train_data['text'][6]))

Whoever wrote the screenplay for this movie obviously never consulted any books about Lucille Ball, especially her autobiography. I've never seen so many mistakes in a biopic, ranging from her early years in Celoron and Jamestown to her later years with Desi. I could write a whole list of factual errors, but it would go on for pages. In all, I believe that Lucille Ball is one of those inimitable people who simply cannot be portrayed by anyone other than themselves. If I were Lucie Arnaz and Desi, Jr., I would be irate at how many mistakes were made in this film. The filmmakers tried hard, but the movie seems awfully sloppy to me.


In [170]:
tokenizer.convert_ids_to_tokens(tokenizer.encode(train_data['text'][6]))[:10]

['[CLS]',
 'whoever',
 'wrote',
 'the',
 'screenplay',
 'for',
 'this',
 'movie',
 'obviously',
 'never']

In [171]:
tokenizer(train_data['text'][6])['input_ids'][:10]

[101, 9444, 2626, 1996, 9000, 2005, 2023, 3185, 5525, 2196]

In [172]:
def tokenize_and_numericalize_data(data, tokenizer):
    ids = tokenizer(data['text'], truncation=True)['input_ids']
    return {'ids': ids}

train_data = train_data.map(lambda data: tokenize_and_numericalize_data(data, tokenizer))
valid_data = valid_data.map(lambda data: tokenize_and_numericalize_data(data, tokenizer))
test_data = test_data.map(lambda data: tokenize_and_numericalize_data(data, tokenizer))

Loading cached processed dataset at /home/sean/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-ec4b64eee868feb1.arrow


  0%|          | 0/6250 [00:00<?, ?ex/s]

  0%|          | 0/18750 [00:00<?, ?ex/s]

We can use the `convert_ids_to_tokens` to transform these indexes back into readable tokens.

## Prepare iterators
we create training and testing iterators. To avoid memory problems during training we will use a small batch size.

In [173]:
def collate(batch, pad_index):
    batch_ids = [torch.tensor(i['ids']) for i in batch]
    batch_ids = nn.utils.rnn.pad_sequence(batch_ids, padding_value=pad_index, batch_first=True)
    batch_label = [torch.tensor(i['label']) for i in batch]
    batch_label = torch.stack(batch_label)
    batch = {'ids': batch_ids,
             'label': batch_label}
    return batch

In [174]:
batch_size = 8


train_dataloader = torch.utils.data.DataLoader(train_data, 
                                               batch_size=batch_size, 
                                               collate_fn=(lambda batch: collate(batch, pad_token_idx)),
                                               shuffle=True)

valid_dataloader = torch.utils.data.DataLoader(valid_data,
                                                batch_size=batch_size,
                                                collate_fn=(lambda batch: collate(batch, pad_token_idx)))

test_dataloader = torch.utils.data.DataLoader(test_data,
                                              batch_size=batch_size,
                                              collate_fn=(lambda batch: collate(batch, pad_token_idx)))

## Build the Model

Next, we'll load the pre-trained model, making sure to load the same model as we did for the tokenizer.

In [175]:
bert = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Now we can define our classification model. 

Instead of using an embedding layer to get embeddings for our text, we'll be using the pre-trained transformer model. These embeddings will then be fed into a GRU to produce a prediction for the sentiment of the input sentence. We get the embedding dimension size (called the `hidden_size`) from the transformer via its config attribute. The rest of the initialisation is standard.

Within the forward pass, we freeze the transformer by using `no_grad` to ensure no gradients are calculated over this part of the model.



In [176]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, ids):
        with torch.no_grad():
            embedded = self.bert(ids)[0]
            
        _, hidden = self.rnn(embedded)
    
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        
        output = self.out(hidden)        
        
        return output

Next, we create an instance of our model using standard hyperparameters.

In [177]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In order to freeze paramers (not train them) we need to set their `requires_grad` attribute to `False`. To do this, we simply loop through all of the `named_parameters` in our model and if they're a part of the `bert` transformer model, we set `requires_grad = False`. 

In [178]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

We can double check the names of the trainable parameters, ensuring none of them belong to BERT. We can see, they are all the parameters of the GRU (`rnn`) and the linear layer (`out`).

In [179]:
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias


## Train the Model

As is standard, we define our optimizer and criterion (loss function).

In [180]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [181]:
criterion = nn.BCEWithLogitsLoss()

Place the model and criterion onto the GPU (if available)

In [182]:
model = model.to(device)
criterion = criterion.to(device)

Next, we'll define functions for: calculating accuracy, performing a training epoch, performing an evaluation epoch and calculating how long a training/evaluation epoch takes.

In [183]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [184]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for batch in tqdm(iterator):
        ids = batch['ids'].to(device)
        labels = batch['label'].to(device)
        optimizer.zero_grad()
        predictions = model(ids).squeeze(1)
        loss = criterion(predictions, labels.float())
        
        acc = binary_accuracy(predictions, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [185]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in tqdm(iterator):
            ids = batch['ids'].to(device)
            labels = batch['label'].to(device)
            optimizer.zero_grad()
            predictions = model(ids).squeeze(1)
            loss = criterion(predictions, labels.float())
            
            acc = binary_accuracy(predictions,  labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [186]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## Training Loop
Finally, we'll train our model. This takes considerably longer than any of the previous models due to the size of the transformer. Even though we are not training any of the transformer's parameters we still need to pass the data through the model which takes a considerable amount of time on a standard GPU.

- To save training time, I uploaded an already trained model for you to use. Please download it from here (https://durhamuniversity-my.sharepoint.com/:u:/g/personal/lkjs74_durham_ac_uk/EecLvOrAhdlDqmkIcFBQmEsBE5lIVwbMkz7LB9lNCmjdxA?e=fIXt1D)


In [187]:
load_saved_model = True # set to True in order to load a previously trained model 

if load_saved_model:
    model.load_state_dict(torch.load('bert-sentiment-model.pt'))

else:
    # Train the model
    N_EPOCHS = 1

    best_valid_loss = float('inf')

    for epoch in (range(N_EPOCHS)):
    
        start_time = time.time()
    
        train_loss, train_acc = train(model, train_dataloader, optimizer, criterion)
        valid_loss, valid_acc = evaluate(model, valid_dataloader, criterion)
        
        end_time = time.time()
        
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), 'bert-sentiment-model.pt')
    
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

## Test the model 

In [188]:
test_loss, test_acc = evaluate(model, test_dataloader, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

100%|██████████| 2344/2344 [02:56<00:00, 13.27it/s]

Test Loss: 0.219 | Test Acc: 91.63%


## Inference

We'll use the model to test the sentiment of some sequences. We tokenize the input sequence, trim it down to the maximum length, add the special tokens to either side, convert it to a tensor, add a batch dimension and then pass it through our model.

In [189]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [190]:
predict_sentiment(model, tokenizer, "This film is terrible")

0.017242131754755974

In [191]:
predict_sentiment(model, tokenizer, "This film is great")

0.9773443341255188

Exercises
=========
-  Try to test the model with different sentences of your choice.   
-  Try with more layers, more hidden units, and more sentences.
-  Replace the GRU with LSTM and analyse the difference in performance.